In [ ]:
import pandas as pd
import requests
from io import StringIO
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.backends.backend_pdf

%matplotlib inline

df_to_export = None
last_fig = None

In [13]:
USDA_URL = "https://fems.fs2c.usda.gov/fuelmodel/sample/download?returnAll=&responseFormat=csv&siteId=All&sampleId=&startDate=2005-01-01T00:00:00.000Z&endDate=2025-12-31T23:00:00.000Z&filterByFuelId=&filterByStatus=Submitted&filterByCategory=All&filterBySubCategory=All&filterByMethod=All&sortBy=fuel_type&sortOrder=asc"

print("Fetching data from USDA ...")
response = requests.get(USDA_URL)
df = pd.read_csv(StringIO(response.text))

df. columns = ["Sample_Id", "Date-Time", "Site Name", "SiteId", 
               "Fuel Type", "Category", "Sub-Category", "Method",
               "SampleAvg Value", "Sample Status"]

df["Date-Time"] = pd.to_datetime(df["Date-Time"], errors="coerce")
df = df[df["Date-Time"].notnull()]
df["Date-Time"] = df["Date-Time"].apply(lambda x: x.replace(tzinfo=None) if x.tzinfo else x)

desired_categories = ["Grass", "Forbs", "Shrubs", "Trees", "Dead"]
existing_categories = df["Category"].dropna().unique().tolist()
missing_categories = [cat for cat in desired_categories if cat not in df['Category'].unique()]

default_date = pd.Timestamp("2005-01-01")
for cat in missing_categories:
    df = pd.concat([df, pd.DataFrame([{
    "Sample_Id": None,
    "Date-Time": default_date,
    "Site Name": "Placeholder Site",
    "SiteId": None,
    "Fuel Type": "Placeholder",
    "Category": cat,
    "Sub-Category": None,
    "Method": None,
    "SampleAvg Value": None,
    "Sample Status": None,
    }])])
    
    
print(f"Data loaded:  {len(df)} rows.")
               
               

Fetching data from USDA ...
Data loaded:  317222 rows.


In [14]:

site_selector = widgets.SelectMultiple(options=sorted(df['Site Name'].dropna().unique()), description='Sites:', layout=widgets.Layout(width='50%'))
category_selector = widgets.SelectMultiple(options=desired_categories, description='Categories:', layout=widgets.Layout(width='50%'))                                     
fuel_selector = widgets.SelectMultiple(options=sorted(df['Fuel Type'].dropna().unique()), description='Fuel Types:', layout=widgets.Layout(width='50%'))
    
years = sorted(df['Date-Time'].dt.year.unique())
months = list(range(1, 13))

current_year = widgets.Dropdown(options=years[::-1], description='Current Year:')
history_years = widgets.SelectMultiple(options=years, description='Historical Years:')
select_all_checkbox = widgets.Checkbox(value=False, description="Use all Historical Years")
overlay_checkbox = widgets.Checkbox(value=True, description="Overlay Historical Data")
month_selector = widgets.SelectMultiple(options=months, value=list(months), description='Months:')

export_button = widgets.Button(description="Export CSV", button_style="success")
pdf_button = widgets.Button(description="Export PDF", button_style='info')
png_button = widgets.Button(description="Export PNG", button_style='warning')

output = widgets.Output()

In [16]:
from datetime import datetime

def update_dashboard(change=None):
    global df_to_export, last_fig
    with output:
        clear_output(wait=True)
        
                  
        filtered_df = df.copy()
        if site_selector.value:
            filtered_df = filtered_df[filtered_df['Site Name'].isin(site_selector.value)]
        if category_selector.value:
            filtered_df = filtered_df[filtered_df['Category'].isin(category_selector.value)]
        if fuel_selector.value:
            filtered_df = filtered_df[filtered_df['Fuel Type'].isin(fuel_selector.value)]
            
        filtered_df['Month'] = filtered_df['Date-Time'].dt.month
        filtered_df = filtered_df[filtered_df['Month'].isin(month_selector.value)]
        filtered_df['Year'] = filtered_df['Date-Time'].dt.year
        
        def to_half_month(dt):
            return datetime(2000, dt.month, 1 if dt.day <=14 else 15)
        filtered_df['AlignDate'] = filtered_df['Date-Time'].apply(to_half_month)
        
        current = filtered_df[filtered_df['Year'] == current_year.value]
        hist_years_selected = list(history_years.option) if select_all_checkbox.value else list(history_years.value)
        hist_years_selected = [y for y in hist_years_selected if y != current_year.value]
        historical = filtered_df[filtered_df['Year'].isin(hist_years_selected)]
        
        if current.empty:
            print("No data available for selected filters or years.")
            return
        
        
        cur_summary = current.groupby(['AlignDate','Category'])["SampleAvg Value"].mean().reset_index(name='Current')
        if overlay_checkbox.value and not historical.empty:
            hist_stats = historical.groupby(['AlignDate','Category'])['SampleAvg Value'].agg(
                Avg='mean', Min='min', Max='max').reset_index()
            
        else:
            hist_stats = pd.DataFrame()
        
        categories = cur_summary['Category'].unique()
        palette = sns.color_palette("tab10", n_colors=len(categories))
        color_map = dict(zip(categories, palette))
        fig, axes = plt.subplots(len(categories), 1, figsize=(16, 5 * len(categories)), sharex=True)
        if len(categories) == 1:
            axes = [axes]
        
        month_ticks = pd.date_range(start='2000-01-01', periods=12, freq='MS')
        
        site_names = ','.join(site_selector.value) if site_selector.value else 'All Sites'
        categories_used = ','.join(category_selector.value) if category_selector.value else 'All Categories'
        fuels_used = ','.join(fuel_selector.value) if fuel_selector.value else 'All Fuel Types'
        
        month_nums = sorted(month_selector.value)
        month_names = [datetime(2000, m, 1).strftime('%b') for m in month_nums]
        if len(month_nums)> 1 and all(b - a == 1 for a,b in zip(month_nums[:-1], month_nums[1:])):
            months_used = f"{month_names[0]}-{month_names[-1]}"
        else:
            months_used = ','.join(month_names)
        
    
        if overlay_checkbox.value and hist_years_selected:
            sorted_years = sorted(hist_years_selected)
            if len(sorted_years) > 1 and all(b - a == 1 for a, b in zip(sorted_years[:1], sorted_years[1:])):
                hist_range = f"{sorted_years[0]}-{sorted_years[-1]}"
            else:
                hist_range = ','.join(map(str, sorted_years))
            years_used = f"{current_year.value} + Hist ({hist_range})"
        else:
            years_used = f"{current_year.value}"
    
        filter_text = f"Filters: {site_names} | {categories_used} | {fuels_used} | Month: {months_used} | Years: {years_used}"
        
        for ax, cat in zip(axes, categories):
            color = color_map[cat]
            cur = cur_summary[cur_summary['Category'] == cat]
            ax.plot(cur['AlignDate'], cur['Current'], marker='o', linestyle='-', label=f"{current_year.value} Current", color=color)
            
            if overlay_checkbox.value and not hist_stats.empty:
                hist = hist_stats[hist_stats['Category'] == cat]
                if 'SingleYear' in hist.columns:
                    ax.plot(hist['AlignDate'], hist['Singleyear'], markers='s', linestyle='--', color='purple', label=f"Hist{hsit_years_selected[0]}")
                else:
                    ax.plot(hist['AlignDate'], hist['Avg'], marker='s', linestyle='--', label="Hist Avg", color='gray', alpha=0.7)
                    ax.plot(hist['AlignDate'], hist['Min'], linestyle=':', color='red', label='Min')
                    ax.plot(hist['AlignDate'], hist['Max'], linestyle=':', color='green', label='Max')
            
            ax.set_title(f"{cat} - Bi-Monthly Comparison\n{filter_text}")
            ax.set_ylabel("Sample Avg Value")
            ax.grid(True)
            ax.legend()
            
        axes[-1].set_xticks(month_ticks)
        axes[-1].set_xticklabels([d.strftime('%b') for d in month_ticks], rotation=0)    
        
        min_month = min(month_selector.value)
        max_month = max(month_selector.value)
        start_date = datetime(2000, min_month, 1)
        end_date = datetime(2000, max_month, 28 if max_month == 2 else 30)
        
        for ax in axes:
            ax.set_xlim(start_date, end_date)
        
        plt.xlabel("Month")
        plt.tight_layout()
        plt.show()
        
        df_to_export = filtered_df.copy()
        last_fig = fig
   


In [17]:
 
    def export_filtered_data(b):
        global df_to_export
        with output:
            if df_to_export is not None:
                df_to_export.to_csv("Filtered_Fuel_Data.csv", index=False)
                print("Data exported to 'Filtered_Fuels_Data.csv'")
            else:
                print("No data to export.")
        
    def export_chart_pdf(b):
        global last_fig
        with output:
            if last_fig:
                pdf = matplotlib.backends.backend_pdf.PdfPages ("Fuel_Sample_Chart.pdf")
                pdf.savefig(last_fig)
                pdf.close()
                print("Chart exported to 'Fuel_Sample_Chart.pdf'")
            else:
                print("No chart to export.")
                
    def export_chart_png(b):
        print("PNG export triggered")
        global last_fig
        with output:
            if last_fig:
                last_fig.savefig("Fuel_Sample_Chart.png", dpi=300)
                print("Chart exported to 'Fuel_Sample_Chart.png'")
            else:
                print("No chart to export.")
        
    

In [18]:
for w in [site_selector, category_selector, fuel_selector, current_year, history_years, overlay_checkbox, month_selector]:
    w.observe(update_dashboard, names='value')
    
export_button.on_click(export_filtered_data)
pdf_button.on_click(export_chart_pdf)
png_button.on_click(export_chart_png)

In [19]:
print("Field Sample dashboard ready.")
display (widgets.HBox([site_selector, category_selector, fuel_selector]))
display(widgets.HBox([current_year, history_years, overlay_checkbox]))
display(widgets.HBox([month_selector]))
display(widgets.HBox([export_button, pdf_button, png_button]))
display(output)

update_dashboard()

Field Sample dashboard ready.


Output()